In [ ]:
import tkinter as tk
from tkinter import messagebox
import heapq


graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('D', 2), ('E', 5)],
    'C': [('F', 3)],
    'D': [('G', 1)],
    'E': [('G', 2)],
    'F': [('H', 5)],
    'G': [('H', 2)],
    'H': []
}

heuristics = {
    'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3,
    'F': 4, 'G': 2, 'H': 0
}

positions = {
    'A': (50, 200), 'B': (150, 150), 'C': (150, 250),
    'D': (250, 100), 'E': (250, 200), 'F': (250, 300),
    'G': (350, 150), 'H': (450, 200)
}

class AStarGame:
    def __init__(self, root):
        self.root = root
        self.root.title("A* Pathfinding Game with Inputs")

        
        self.canvas = tk.Canvas(root, width=500, height=400, bg="white")
        self.canvas.grid(row=0, column=0, columnspan=4, padx=10, pady=10)
        self.draw_graph()

        
        tk.Label(root, text="Start:").grid(row=1, column=0)
        self.start_entry = tk.Entry(root, width=5)
        self.start_entry.grid(row=1, column=1)

        tk.Label(root, text="End:").grid(row=1, column=2)
        self.end_entry = tk.Entry(root, width=5)
        self.end_entry.grid(row=1, column=3)

        tk.Button(root, text="Find Path", command=self.find_path).grid(row=2, column=0, columnspan=4, pady=5)

        
        tk.Label(root, text="From:").grid(row=3, column=0)
        self.edge_from = tk.Entry(root, width=5)
        self.edge_from.grid(row=3, column=1)

        tk.Label(root, text="To:").grid(row=3, column=2)
        self.edge_to = tk.Entry(root, width=5)
        self.edge_to.grid(row=3, column=3)

        tk.Label(root, text="Cost:").grid(row=4, column=0)
        self.edge_cost = tk.Entry(root, width=5)
        self.edge_cost.grid(row=4, column=1)

        tk.Button(root, text="Add/Update Edge", command=self.update_edge).grid(row=4, column=2, columnspan=2)

        
        tk.Label(root, text="Node:").grid(row=5, column=0)
        self.heuristic_node = tk.Entry(root, width=5)
        self.heuristic_node.grid(row=5, column=1)

        tk.Label(root, text="h(n):").grid(row=5, column=2)
        self.heuristic_value = tk.Entry(root, width=5)
        self.heuristic_value.grid(row=5, column=3)

        tk.Button(root, text="Set Heuristic", command=self.update_heuristic).grid(row=6, column=0, columnspan=4, pady=5)

    def draw_graph(self):
        self.canvas.delete("all")
        for node, (x, y) in positions.items():
            self.canvas.create_oval(x-20, y-20, x+20, y+20, fill="lightblue", tags=node)
            self.canvas.create_text(x, y, text=node, font=("Arial", 14, "bold"))

        for src in graph:
            for dest, _ in graph[src]:
                if src in positions and dest in positions:
                    x1, y1 = positions[src]
                    x2, y2 = positions[dest]
                    self.canvas.create_line(x1, y1, x2, y2, arrow=tk.LAST)

    def highlight_path(self, path):
        for i in range(len(path) - 1):
            if path[i] in positions and path[i + 1] in positions:
                x1, y1 = positions[path[i]]
                x2, y2 = positions[path[i + 1]]
                self.canvas.create_line(x1, y1, x2, y2, width=4, fill="orange", arrow=tk.LAST)

    def find_path(self):
        start = self.start_entry.get().strip().upper()
        end = self.end_entry.get().strip().upper()

        if start not in graph or end not in graph:
            messagebox.showerror("Invalid Node", "Start or End node doesn't exist.")
            return

        path, cost = self.a_star(start, end)
        if path:
            self.draw_graph()
            self.highlight_path(path)
            messagebox.showinfo("Path Found", f"Path: {' -> '.join(path)}\nCost: {cost}")
        else:
            messagebox.showwarning("No Path", "Could not find a path.")

    def update_edge(self):
        src = self.edge_from.get().strip().upper()
        dst = self.edge_to.get().strip().upper()

        try:
            cost = int(self.edge_cost.get().strip())
        except ValueError:
            messagebox.showerror("Invalid Input", "Cost must be an integer.")
            return

        if not src or not dst:
            messagebox.showerror("Missing Input", "Please enter both source and destination nodes.")
            return

        if src not in graph:
            graph[src] = []

        updated = False
        for i, (node, _) in enumerate(graph[src]):
            if node == dst:
                graph[src][i] = (dst, cost)
                updated = True
                break

        if not updated:
            graph[src].append((dst, cost))

        if dst not in graph:
            graph[dst] = []

        self.draw_graph()

        if updated:
            messagebox.showinfo("Edge Updated", f"Updated edge: {src} → {dst} with cost {cost}")
        else:
            messagebox.showinfo("Edge Added", f"Added edge: {src} → {dst} with cost {cost}")

    def update_heuristic(self):
        node = self.heuristic_node.get().strip().upper()
        try:
            value = int(self.heuristic_value.get().strip())
        except ValueError:
            messagebox.showerror("Invalid Input", "Heuristic must be an integer.")
            return

        heuristics[node] = value
        messagebox.showinfo("Heuristic Set", f"Set h({node}) = {value}")

    def a_star(self, start, goal):
        open_list = []
        heapq.heappush(open_list, (heuristics.get(start, 0), start, [start], 0))
        visited = set()

        while open_list:
            f, current, path, g = heapq.heappop(open_list)
            if current in visited:
                continue
            visited.add(current)
            if current == goal:
                return path, g
            for neighbor, cost in graph.get(current, []):
                if neighbor not in visited:
                    g_new = g + cost
                    f_new = g_new + heuristics.get(neighbor, 0)
                    heapq.heappush(open_list, (f_new, neighbor, path + [neighbor], g_new))
        return None, float('inf')



if __name__ == "__main__":
    root = tk.Tk()
    app = AStarGame(root)
    root.mainloop()
